Семинар 1

In [1]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

from nltk.tokenize import word_tokenize 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import eli5

In [2]:
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding='utf8', sep='\t')

In [3]:
df['sentiment'].value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [4]:
# так как задача бинарной классификации, уберём класс neutral
df=df[df['sentiment'] != 'neautral'] 

In [5]:
# посмотрим на пример отзыва
df.iloc[0]['review']
# нужна предобработка текста: убрать знаки препинания и привести к одному регистру

'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

In [6]:
# убрали знаки препинания (регулярное выражение заменяет знаки на пустой символ)
# чтобы ничего случайно не затереть, запишем в отдельную колонку review_processed
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values

In [7]:
# привели к нижнему регистру
df['review_processed'] = df['review_processed'].str.lower()

In [8]:
# токенизация 
df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))

100%|██████████| 60000/60000 [00:06<00:00, 9148.25it/s]


In [9]:
# результат
df['review_processed'].iloc[0]

['качество',
 'плохое',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не',
 'соответствует',
 'ткань',
 'ужасная',
 'рисунок',
 'блеклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'ужас',
 'не',
 'стоит',
 'за',
 'такие',
 'деньги',
 'г']

In [10]:
morph = pymorphy2.MorphAnalyzer()

In [11]:
# приводим к нормальной форме
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])

100%|██████████| 60000/60000 [02:53<00:00, 345.54it/s]


In [12]:
# результат
df['review_lemmatized'].iloc[0]

['качество',
 'плохой',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не',
 'соответствовать',
 'ткань',
 'ужасный',
 'рисунок',
 'блёклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'ужас',
 'не',
 'стоить',
 'за',
 'такой',
 'деньга',
 'г']

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [14]:
# векторизация
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))

In [15]:
X.shape

(60000, 51549)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

In [17]:
logit = LogisticRegression()

In [18]:
logit.fit(X_train, y_train)

LogisticRegression()

In [19]:
y_pred = logit.predict_proba(X_test)[:, 1]
y_pred

array([0.18457169, 0.6284437 , 0.99264536, ..., 0.25177362, 0.00206928,
       0.00343717])

In [20]:
roc_auc_score(y_test, y_pred)

0.9735222897024753

Семинар 2 (продолжение)

In [21]:
# визулизация весов
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names_out()), top=(50,50))

Weight?,Feature
+8.682,отличный
+7.421,хороший
+7.047,супер
+6.590,немного
+6.107,отлично
+5.634,хорошо
+5.467,спасибо
+5.367,приятный
+5.291,классный
+5.009,мягкий


In [22]:
# коллокации (1 способ)
vectorizer = TfidfVectorizer(ngram_range=(1, 2)) # униграммы и биграммы
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))
X.shape
# матрица стала больше, поскольку здесь не только слава, но и сочетания из 2-х слов

(60000, 396100)

In [23]:
# снова обучим логистичекую регрессию
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.9777779791573166

In [24]:
# появились словосочетания
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names_out()), top=(15,10))

Weight?,Feature
+10.460,отличный
+10.075,хороший
+7.917,супер
+7.339,спасибо
+7.282,немного
+6.904,хорошо
+6.754,отлично
+6.230,приятный
+6.177,классный
+6.061,довольный


In [25]:
# коллокации (2 способ)
from gensim.models import Phrases
documents = ["Нью Йорк большой город", "Нью Йорк", "Поеду в Нью Йорк на следующей неделе"]

# разбить строку на список слов
sentence_stream = [doc.split(" ") for doc in documents]
# если какое-то сочетание слов повторяется больше, чем 2 раза, то это коллокация
bigram = Phrases(sentence_stream, min_count=2, threshold=1)
# предложение для поиска коллокаций
sent = ['Он', 'поехал', 'в', 'Нью', 'Йорк']
print(bigram[sent])

['Он', 'поехал', 'в', 'Нью_Йорк']


In [26]:
# обучим на датасете
bigram = Phrases(df['review_lemmatized'].values, min_count=2, threshold=7)

In [27]:
# вывести найденные колокации
bigram.export_phrases()

{'за_такой': 10.687130891423308,
 'другой_человек': 26.485106382978724,
 'ладный_хоть': 107.42918079405008,
 'деньга_вернуть': 8.553132410577003,
 'ничего_общий': 8.455688064859672,
 'на_картинка': 8.417227527576905,
 'сам_дело': 94.92785411846185,
 'продлить_защита': 151.28443495863405,
 'мой_согласие': 57.30904895772556,
 'голый_синтетик': 96.06008084671723,
 'понимать_почему': 88.75702990309732,
 'в_остальной': 9.049935185623038,
 'за_общительность': 12.163777089783283,
 'размер_s': 8.535167916211655,
 'тот_что': 10.09385892045114,
 'постоянно_сползать': 122.84210526315789,
 'капроновый_колготки': 298.9603842678461,
 'после_первый': 21.351067688588756,
 'первый_носка': 9.65976312448307,
 'сразу_же': 25.083938961791304,
 'до_граница': 14.538378991772847,
 'подозрение_что': 16.582768226455443,
 'продавец_отказываться': 8.512525582642933,
 'отказываться_возвращать': 157.54556722076407,
 'отправить_повторно': 36.29496762869614,
 'последний_раз': 8.909318234995201,
 'обещать_продлить': 1

In [28]:
# добавим столбец коллокаций
df['review_lemmatized_coll'] = df['review_lemmatized'].progress_apply(lambda x: bigram[x])

100%|██████████| 60000/60000 [00:02<00:00, 28752.43it/s]


In [29]:
# TF_IDF на коллокациях
vectorizer = TfidfVectorizer(ngram_range=(1, 1)) # униграммы и биграммы
X = vectorizer.fit_transform(df['review_lemmatized_coll'].apply(lambda x: ' '.join(x)))
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)
# качество немного ухудшилось

0.971785345911664

In [30]:
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names_out()), top=(15,10))

Weight?,Feature
+8.923,отличный
+7.736,хороший
+7.026,супер
+6.193,немного
+5.742,спасибо
+5.499,классный
+5.452,отлично
+5.146,хорошо
+4.750,мягкий
+4.550,довольный


In [31]:
# подбор гиперпараметров по сетке
import warnings
from sklearn.exceptions import ConvergenceWarning

# Игнорировать предупреждения о сходимости
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    parameters = {'C':[0.5, 1, 10, 20],
              'max_iter': [50, 100, 300]}
    clf = GridSearchCV(logit, parameters)
    clf.fit(X_train, y_train)

In [32]:
clf.best_params_

{'C': 10, 'max_iter': 100}